In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_validate, train_test_split

## 패션 MNIST

In [14]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [15]:
lables = np.array(["티셔츠", "바지", "스웨터", "드레스", "코트", "샌달", "셔츠", "스니커즈", "가방", "앵클 부츠"])
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)
print(lables[train_target[:5]])
print(train_target[:5])

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)
['앵클 부츠' '티셔츠' '티셔츠' '드레스' '티셔츠']
[9 0 0 3 0]


이미지 각 픽셀 값을 0 ~ 1로 정규화

In [16]:
train_scaled = train_input / 255 # 이미지 픽셀 최대 255
train_scaled2 = train_scaled.reshape(-1, 28 * 28) # 1차원 배열로 재구성
print(train_scaled.shape, train_scaled2.shape)

(60000, 28, 28) (60000, 784)


### 확률적 경사 하강법 클래스 사용

In [17]:
# sc = SGDClassifier(loss="log_loss", max_iter = 5, random_state = 42)
# scores = cross_validate(sc, train_scaled2, train_target, n_jobs=-1) # 교차 검증 (모델, 학습 데이터, 타깃 데이터, 병렬처리)
# print(np.mean(scores['test_score']))
# print(scores)

In [18]:
# sc = SGDClassifier(loss="log_loss", max_iter = 9, random_state = 42)
# scores = cross_validate(sc, train_scaled2, train_target, n_jobs=-1) # 교차 검증
# print(np.mean(scores['test_score']))
# print(scores)

In [19]:
# sc = SGDClassifier(loss="log_loss", max_iter = 20, random_state = 42)
# scores = cross_validate(sc, train_scaled2, train_target, n_jobs=-1) # 교차 검증
# print(np.mean(scores['test_score']))
# print(scores)

## 검증 셋 분리

In [20]:
train_scaled3, val_scaled, train_target, val_target = train_test_split(train_scaled2, train_target, test_size = 0.2 , random_state = 42)

print(train_scaled3.shape, train_target.shape)
print(val_scaled.shape, val_target.shape)

(48000, 784) (48000,)
(12000, 784) (12000,)


## 밀집층 구성 밎 모델 만들기

In [23]:
# 패션 아이템이 10개라서 뉴런 10개
dense = keras.layers.Dense(10, activation='softmax', input_shape = (784, ))
model = keras.Sequential(dense)

In [25]:
model.compile(loss = "sparse_categorical_crossentropy", metrics = "accuracy")

sparse가 붙은 이유는 타깃값을 원-핫 인코딩으로 부꾸지 않고 사용하기 위해서임

In [27]:
print(train_target[:10])
print(lables[train_target[:10]])

[7 3 5 8 6 9 3 3 9 9]
['스니커즈' '드레스' '샌달' '가방' '셔츠' '앵클 부츠' '드레스' '드레스' '앵클 부츠' '앵클 부츠']


In [28]:
# 모델 훈련
model.fit(train_scaled3, train_target, epochs = 5)
# 미니 배치 경사 하강법을 사용하기 때문에 4만8천개의 데이터들을 32개씩 묶어서 학습하기 때문에 1500이란 숫자가 나옴

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.6092 - accuracy: 0.7932
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4755 - accuracy: 0.8386
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4521 - accuracy: 0.8472
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4385 - accuracy: 0.8507
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4307 - accuracy: 0.8542


In [29]:
#평가
model.evaluate(val_scaled, val_target)
# 에포크 5회 수행 결과 -> 훈련 셋 점수 : 85, 검증셋 점수 84

375/375 [==============================] - 1s 1ms/step - loss: 0.4455 - accuracy: 0.8469


[0.4454647898674011, 0.846916675567627]